<a href="https://colab.research.google.com/github/Kuyas/ImageSplicing/blob/master/Forgery_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import resnet152
import torch.optim as optim
from torch.autograd import Variable
from sklearn import svm
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2
from google.colab.patches import cv2_imshow
from torchvision import datasets
import statistics

In [ ]:
EPOCHS = 50
BATCH_SIZE = 2
LEARNING_RATE = 0.3
TRAIN_DATAPATH = '/content/drive/My Drive/CTScan/run_2/train/'
VAL_DATAPATH = '/content/drive/My Drive/CTScan/run_2/test/'
TEST_DATAPATH = '/content/drive/My Drive/CTScan/run_2/test/'

In [ ]:
model = models.resnet152(True)
print(model)
for param in model.parameters():
    param.requires_grad = False

# model.fc[-1] = nn.Linear(1000, 256)
# print(model) 
model.fc = nn.Sequential(nn.Linear(2048, 1000), 
                  nn.ReLU(), 
                  nn.Dropout(0.8),
                  nn.Linear(1000, 2))

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]        
    print(len(weight))                          
    return weight      



In [ ]:
TRAIN_TRANSFORM_IMG = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees = 15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
        ])

VAL_TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATAPATH, transform=TRAIN_TRANSFORM_IMG)

weights = make_weights_for_balanced_classes(train_data.imgs, len(train_data.classes))
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=0,sampler = sampler,pin_memory=True)
val_data = torchvision.datasets.ImageFolder(root=VAL_DATAPATH, transform=VAL_TRANSFORM_IMG)
val_data_loader = torch.utils.data.DataLoader(val_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=VAL_TRANSFORM_IMG)
test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)

128


In [ ]:
dataiter = iter(train_data_loader)
images, labels = dataiter.next()
print("Number of Training Examples: ", len(train_data))
print("Number of Test Examples: ", len(test_data))
print("Number of Validation Examples: ", len(val_data))
print("Detected Classes are: ", train_data.class_to_idx)
train_iter = iter(train_data_loader)
images, labels = train_iter.next()
print("Image Shape on Batch size = {} ".format(images.size()))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Number of Training Examples:  128
Number of Test Examples:  35
Number of Validation Examples:  35
Detected Classes are:  {'tampered': 0, 'true': 1}
Image Shape on Batch size = torch.Size([2, 3, 224, 224]) 
cuda:0


In [ ]:
model.to(device)
weight_c = torch.Tensor([1.34,3.87]).cuda()
criterion = nn.NLLLoss()
_params = filter(lambda p: p.requires_grad, model.parameters())
# optimizer = optim.Adam(_params, lr=LEARNING_RATE, weight_decay=1e-5)
optimizer = optim.SGD(_params, lr=0.003, momentum=0.9)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

for epoch in range(EPOCHS):
    start = time.time()
    correct = 0 
    iterations = 0
    iter_loss = 0.0

    model.train()

    for i,data in enumerate(train_data_loader,0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        iter_loss+=loss.item()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum()
        iterations += 1

    train_loss.append(iter_loss/iterations)
    # Record the training accuracy
    train_accuracy.append((100 * correct / len(train_data)))

    loss = 0.0
    correct = 0
    iterations = 0
    model.eval()

    for i, data in enumerate(val_data_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)     
        loss = criterion(outputs, labels)
        loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum()

        iterations+=1

    test_loss.append(loss/iterations)
    # Record the Testing accuracy
    test_accuracy.append((100 * correct / len(val_data)))
    stop = time.time()
    print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}, Validation Loss: {:.3f}, Validation Acc: {:.3f}, Time: {:.3f}s'
            .format(epoch+1, EPOCHS, train_loss[-1], train_accuracy[-1], test_loss[-1], test_accuracy[-1], stop-start))

Epoch 1/50, Training Loss: -614226247715670851584.000, Training Accuracy: 50.000, Validation Loss: -2713600889800655634432.000, Validation Acc: 48.000, Time: 3.553s
Epoch 2/50, Training Loss: -5692578356582206322996076871680.000, Training Accuracy: 49.000, Validation Loss: -9421878827582385670158767095808.000, Validation Acc: 51.000, Time: 3.548s
Epoch 3/50, Training Loss: -inf, Training Accuracy: 55.000, Validation Loss: -inf, Validation Acc: 51.000, Time: 3.560s
Epoch 4/50, Training Loss: -inf, Training Accuracy: 43.000, Validation Loss: -inf, Validation Acc: 51.000, Time: 3.536s
Epoch 5/50, Training Loss: -inf, Training Accuracy: 45.000, Validation Loss: -inf, Validation Acc: 51.000, Time: 3.541s
Epoch 6/50, Training Loss: -inf, Training Accuracy: 40.000, Validation Loss: -inf, Validation Acc: 51.000, Time: 3.529s
Epoch 7/50, Training Loss: nan, Training Accuracy: 50.000, Validation Loss: nan, Validation Acc: 51.000, Time: 3.576s
Epoch 8/50, Training Loss: nan, Training Accuracy: 46

In [ ]:
correct = 0
total = 0
classes =['tampered','normal']
confusion_matrix = torch.zeros(2, 2)
correct_0 = 0
correct_1 = 0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in test_data_loader:
        images,labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        c = (predicted == labels).squeeze()
        for i in range(2):
            try:
              label = labels[i]
              # c1 = c.cpu().data.numpy()
              class_correct[label] += c[i].item()
              class_total[label] += 1
            except:
              continue
        correct+=(predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
acc = 100*correct/total
print(correct)
print(total)
print('Testing accuracy: ' + str(acc))
print(confusion_matrix)

for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
acc0 = 100 * class_correct[0] / class_total[0]
acc1 = 100 * class_correct[1] / class_total[1]
print("Testing accuracy",acc)
print("Accuracy of normal",acc1,class_correct[1],class_total[1]-class_correct[1])
print("Accuracy of tampered",acc0,class_correct[0],class_total[0]-class_correct[0])


18
35
Testing accuracy: 51.42857142857143
tensor([[18.,  0.],
        [17.,  0.]])
Accuracy of tampered : 100 %
Accuracy of normal :  0 %
Testing accuracy 51.42857142857143
Accuracy of normal 0.0 0.0 17.0
Accuracy of tampered 100.0 17.0 0.0


In [ ]:
tensor([[ 5., 13.],
        [10.,  7.]])